## Notebook for making predictions with deleted constant y samples


In [1]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

### Reading dataset stored in other notebooks

In [3]:
sys.path.append('../')

%store -r dml

dm = dml

## Getting data

In [31]:
X_A_train = dm.train_a.iloc[:, 2:-1]
Y_A_train = dm.train_a.iloc[:, 0]

X_B_train = dm.train_b.iloc[:, 2:-1]
Y_B_train = dm.train_b.iloc[:, 0]

X_C_train = dm.train_c.iloc[:, 2:-1]
Y_C_train = dm.train_c.iloc[:, 0]

X_A_test = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_test = dm.X_test_estimated_b
X_C_test = dm.X_test_estimated_c

In [20]:
# Concat for common b and c model

X_BC_train = pd.concat([X_B_train, X_C_train])
Y_BC_train = pd.concat([Y_B_train, Y_C_train])

## Catboost models
will try with two models:
-   One for dataset A
-   One for B and C

In [21]:
catModel_a = CatBoostRegressor()
catModel_bc = CatBoostRegressor()

In [22]:
catModel_a.fit(X_A_train,Y_A_train)
catModel_bc.fit(X_BC_train,Y_BC_train)

Learning rate set to 0.088994
0:	learn: 1071.2875634	total: 173ms	remaining: 2m 52s
1:	learn: 997.4273049	total: 189ms	remaining: 1m 34s
2:	learn: 931.7008697	total: 201ms	remaining: 1m 6s
3:	learn: 872.7550691	total: 210ms	remaining: 52.3s
4:	learn: 819.8117336	total: 219ms	remaining: 43.5s
5:	learn: 773.2529212	total: 227ms	remaining: 37.7s
6:	learn: 731.0390629	total: 236ms	remaining: 33.5s
7:	learn: 694.7365637	total: 244ms	remaining: 30.3s
8:	learn: 662.3506630	total: 252ms	remaining: 27.8s
9:	learn: 633.3892813	total: 260ms	remaining: 25.7s
10:	learn: 608.5341913	total: 268ms	remaining: 24.1s
11:	learn: 586.3831378	total: 279ms	remaining: 23s
12:	learn: 567.0264412	total: 288ms	remaining: 21.9s
13:	learn: 550.2192699	total: 297ms	remaining: 20.9s
14:	learn: 535.4348890	total: 305ms	remaining: 20.1s
15:	learn: 522.6611491	total: 315ms	remaining: 19.4s
16:	learn: 511.5297849	total: 323ms	remaining: 18.7s
17:	learn: 501.3900267	total: 341ms	remaining: 18.6s
18:	learn: 492.8534268	to

## Preds

In [32]:
X_A_test.head(1)

,absolute_humidity_2m:gm3,ceiling_height_agl:m,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,fresh_snow_1h:cm,fresh_snow_24h:cm,is_in_shadow:idx,msl_pressure:hPa,precip_type_5min:idx,...,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_melt_10min:mm,snow_water:kgm2,sun_elevation:d,total_cloud_cover:p,visibility:m,wind_speed_10m:ms
0,4.15,2969.787598,1236.5,0.0,271.024994,0.0,0.0,1.0,1013.775024,0.0,...,0.0,0.0,80.949997,1010.575012,0.0,0.0,-10.578,69.300003,31241.974609,3.425


In [25]:
pred_A = catModel_a.predict(X_A_test)
pred_B = catModel_bc.predict(X_B_test)
pred_C = catModel_bc.predict(X_C_test)

CatBoostError: C:/Go_Agent/pipelines/BuildMaster/catboost.git/catboost/libs/data/model_dataset_compatibility.cpp:81: At position 1 should be feature with name air_density_2m:kgm3 (found ceiling_height_agl:m).

In [56]:
df_A = pd.DataFrame()

df_A["prediction"] = pred_A
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["prediction"] = pred_B
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["prediction"] = pred_C
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)

df = df.drop("location", axis=1)

# NAME THE FILE 
df.to_csv("sub32.csv")

df.shape

(2160, 1)